In [1]:
import urllib.parse
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup
import validators
import requests
import warnings
import threading

warnings.filterwarnings('ignore')

In [2]:
def get_full_link(url: str, base_url: str) -> str:
    domain = urlparse(base_url).netloc
    if url:
        if not url.startswith("http"):   
            return urljoin(base_url, url)
        elif domain not in url:              
            return
        else:
            return url

In [3]:
lock = threading.Lock()

def get_links(url: str, base_url: str) -> list[str]:
    if url:
        try:
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, "html.parser")
        except Exception as error:
            print(error)
            return
        links = []
        for element in soup.find_all(name=['a', 'link']):
            link = element.get("href")
            link = get_full_link(link, base_url)
            if link:
                links.append(link)
        return links

In [4]:
def worker(url: str, chank_size: int) -> None:
    global checked
    global not_checked
    global in_work

    while not_checked or in_work>0:
        if not not_checked:
            continue

        in_work += 1

        with lock:
            chank = []
            for _ in range(chank_size):
                if not_checked == []:
                    break
                chank.append(not_checked.pop(0))

        for link in chank:
            if link:
                link = get_full_link(link, url)
                new_links = get_links(link, url)
                if new_links:
                    for i in new_links:
                        if not i in not_checked and not i in checked and validators.url(i):
                            not_checked.append(i)
                if link not in checked:
                    checked.append(link)   

        in_work -= 1   


In [5]:
site = "http://crawler-test.com/"
checked = [site]
not_checked = get_links(site, site)
in_work = 0
chank_size = 20
threads = []
for n in range(4):
    t = threading.Thread(target=worker, args=(site, 20))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Exceeded 30 redirects.
No connection adapters were found for 'relativeurl:withcolon'
Exceeded 30 redirects.
HTTPSConnectionPool(host='invalid.crawler-test.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000014C115B76A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Exceeded 30 redirects.
Exceeded 30 redirects.


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


HTTPSConnectionPool(host='crawler-test.com', port=80): Max retries exceeded with url: /canonical_tags/canonical_port_80 (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:997)')))
HTTPSConnectionPool(host='crawler-test.com', port=8080): Max retries exceeded with url: /canonical_tags/canonical_port_8080 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014C113B6BF0>, 'Connection to crawler-test.com timed out. (connect timeout=None)'))
HTTPSConnectionPool(host='crawler-test.com', port=80): Max retries exceeded with url: /canonical_tags/canonical_port_80 (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:997)')))
HTTPSConnectionPool(host='crawler-test.com', port=8080): Max retries exceeded with url: /canonical_tags/canonical_port_8080 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014C1AC70D60>, 'Connection to crawler-test.com timed out. (conne

In [9]:
checked

['http://crawler-test.com/',
 'http://crawler-test.com/encoding/url/encoded_hashbang%23abc',
 'http://crawler-test.com/css/app.css',
 'http://crawler-test.com/description_tags/no_description_nosnippet',
 'http://crawler-test.com/mobile/separate_desktop_with_different_links_out',
 'http://crawler-test.com/titles/title_with_whitespace',
 'http://crawler-test.com/favicon.ico?r=1.6',
 'http://crawler-test.com/mobile/separate_desktop_with_mobile_not_subdomain',
 'http://crawler-test.com/description_tags/duplicate_description',
 'http://crawler-test.com/titles/empty_title',
 'http://crawler-test.com/mobile/desktop_with_self_canonical_mobile_and_amp',
 'http://crawler-test.com/description_tags/duplicate_description/foo',
 'http://crawler-test.com/titles/missing_title',
 'http://crawler-test.com/description_tags/duplicate_description_and_noindex',
 'https://m.crawler-test.com',
 'http://crawler-test.com/mobile/separate_mobile_with_mobile_not_subdomain',
 'http://crawler-test.com/titles/duplica